Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default_backtest
```

## TODO

* Wrap all Exchange Order responses in an Order object
* Verify starting_cash <= actual balance on Live Exchange
* Handle/prevent 'insufficient funds' from Binance/Poloniex (Above will help, but still need to catch if the exchange balance has changed for some reason)
* Handle/prevent 'minimum order not met' from Binanace/Poloniex (method to refresh and lookup the minimum amount)
* Method to audit/confirm our local exchange balance reflects actual changes to CCXT exchange balance
* Limit Sell Orders not working I think..


## Define Strategy

In [5]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.ETH, c.BTC)
        price = data['close']
        quantity = .01

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, price, quantity)
        else:
            order = order_manager.build_market_sell_order(
                ctx.exchange, asset, quantity)
        print("ORDER", order)
        orders.append(order)

        # Optionally cancel pending orders (LIVE trading)
        #pending_orders = ctx.exchange.fetch_open_orders(asset)
        cancel_ids = []
        
        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return {
            'orders': orders,
            'cancel_ids': cancel_ids
        }

## Backtest

In [6]:
cash = 1.0
ohlcv_fpath = os.path.join(cfg.DATA_DIR, c.DEFAULT_30M_FEED_CSV_FILENAME)
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.THIRTY_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
feed = CSVDataFeed(
    fpath=ohlcv_fpath,
    start=None, # Usually None for backtest, but its possible to filter the csv
    end=None
)
exchange = load_feed_based_paper_exchange(balance, feed)
strategy = SimpleStrategy()
experiment_name = 'default_backtest'

In [7]:
runner.backtest(experiment_name, exchange, balance, portfolio, feed, strategy)

---------------------------------------
Epoch 1 - Timestep: 2018-01-09T00:00:00
---------------------------------------
OHLCV
    O: 0.0760 | C: 0.0766 | V: 7900.8 | T: 2018-01-09T00:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 2 - Timestep: 2018-01-09T00:30:00
---------------------------------------
OHLCV
    O: 0.0766 | C: 0.0774 | V: 7715.3 | T: 2018-01-09T00:30:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0774 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0000 Total Val: 1.0000 PnL: 0.0000 Returns: 0.0000
BALANCE
    BTC - {'free': 1.0, 'used': 0.0, 'total': 0.0}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 3 - Timestep: 2018-01-0

ORDER {
    "id": "0cb0f16618504e5fbef9d16a90d23026",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": null,
    "quantity": 1.0,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-17T06:08:21.023313",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "symbol": "ETH/BTC"
}
Insufficient funds to place order 0cb0f16618504e5fbef9d16a90d23026,                     cancelling ...
ORDER {
    "id": "8af2455394194ecb96791516ca6912f4",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": 0.07740599999999999,
    "quantity": 1.0,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "CREATED",
    "created_time": "2018-01-17T06:08:21.074145",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "symbol": "ETH/BTC"
}
Order Places Response {'id': '74fc

---------------------------------------
Epoch 5 - Timestep: 2018-01-09T02:00:00
---------------------------------------
OHLCV
    O: 0.0765 | C: 0.0773 | V: 10995.4 | T: 2018-01-09T02:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0773 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.7693 Total Val: 0.9985 PnL: -0.0015 Returns: -0.0015
BALANCE
    BTC - {'free': 0.53853200000000001, 'used': 0.0, 'total': 0.53853200000000001}
    ETH - {'free': 6.0, 'used': 0.0, 'total': 6.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 3.0, 'cost_price': 0.076911333333333332, 'latest_price': 0.076405000000000001}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 6 - Timestep: 2018-01-09T02:30:00
---------------------------------------
OHLCV
    O: 0.0772 | C: 0.0780 | V: 8033.3 | T: 2018-01-09T02:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.6920 Total Val: 1.0010 PnL: 0.0010 Returns: 0.0010
BALANCE
    BTC - {'fre

Order Places Response {'id': 'fd9e73f4b870478db6ec282c87a83676', 'symbol': 'ETH/BTC', 'price': 0.077252000000000001, 'quantity': 1.0, 'type': 'limit', 'side': 'buy', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:21.283234'}
Order dictionary {'id': 'fd9e73f4b870478db6ec282c87a83676', 'symbol': 'ETH/BTC', 'price': 0.077252000000000001, 'quantity': 1.0, 'type': 'limit', 'side': 'buy', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:21.283234'}
Order fetched {
    "id": null,
    "exchange_id": "paper",
    "exchange_order_id": "fd9e73f4b870478db6ec282c87a83676",
    "price": 0.077252,
    "quantity": 1.0,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-17T06:08:21.283234",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "symbol": "ETH/BTC"
}
Equal? True
ORDER {
    "id": "4fbd26a27bce43dea4909ee3

---------------------------------------
Epoch 7 - Timestep: 2018-01-09T03:00:00
---------------------------------------
OHLCV
    O: 0.0780 | C: 0.0785 | V: 8968.9 | T: 2018-01-09T03:00:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.7808 Total Val: 1.0472 PnL: 0.0472 Returns: 0.0472
BALANCE
    BTC - {'free': 0.56160600000000005, 'used': 0.0, 'total': 0.56160600000000005}
    ETH - {'free': 6.0, 'used': 0.0, 'total': 6.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 3.0, 'cost_price': 0.076996499999999996, 'latest_price': 0.088789}
METRICS
    SMA: 5.00
    RSI: 10.00


...

Order Places Response {'id': '488c794e808c42d3b10de7bf6dda8db2', 'symbol': 'ETH/BTC', 'price': 0.0888, 'quantity': 1.0, 'type': 'market', 'side': 'sell', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:41.456267'}
Order dictionary {'id': '488c794e808c42d3b10de7bf6dda8db2', 'symbol': 'ETH/BTC', 'price': 0.0888, 'quantity': 1.0, 'type': 'market', 'side': 'sell', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:41.456267'}
Order fetched {
    "id": null,
    "exchange_id": "paper",
    "exchange_order_id": "488c794e808c42d3b10de7bf6dda8db2",
    "price": 0.0888,
    "quantity": 1.0,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "FILLED",
    "created_time": "2018-01-17T06:08:41.456267",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "symbol": "ETH/BTC"
}
Equal? True
ORDER {
    "id": "3628a8e632ca41bf894b9989937e4ca1",
    "exchange

---------------------------------------
Epoch 145 - Timestep: 2018-01-16T00:30:00
---------------------------------------
OHLCV
    O: 0.0938 | C: 0.0935 | V: 2213.2 | T: 2018-01-16T00:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9822 Total Val: 1.0710 PnL: 0.0710 Returns: 0.0710
BALANCE
    BTC - {'free': 0.96445999999999987, 'used': 0.0, 'total': 0.96445999999999987}
    ETH - {'free': 2.0, 'used': 0.0, 'total': 2.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 1.0, 'cost_price': 0.087482333333333329, 'latest_price': 0.0888}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 146 - Timestep: 2018-01-16T02:00:00
---------------------------------------
OHLCV


ORDER {
    "id": "b9ebcde4285446a0bff8c80dc25b6d11",
    "exchange_id": "paper",
    "exchange_order_id": null,
    "price": null,
    "quantity": 1.0,
    "filled_quantity": 0.0,
    "order_type": "MARKET_SELL",
    "status": "CREATED",
    "created_time": "2018-01-17T06:08:41.673084",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "symbol": "ETH/BTC"
}
Order Places Response {'id': 'a27a057880a3491ea0679c732d4106a4', 'symbol': 'ETH/BTC', 'price': 0.0888, 'quantity': 1.0, 'type': 'market', 'side': 'sell', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:41.852493'}
Order dictionary {'id': 'a27a057880a3491ea0679c732d4106a4', 'symbol': 'ETH/BTC', 'price': 0.0888, 'quantity': 1.0, 'type': 'market', 'side': 'sell', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:41.852493'}
Order fetched {
    "id": null,
    "exchange_id": "paper",
    "exchange_order_id": "a2

    O: 0.0941 | C: 0.0944 | V: 4164.2 | T: 2018-01-16T02:00:00
ORDERS
    1: ETH/BTC | LIMIT_BUY | Price: 0.0944 | Quantity: 1.0000
PERFORMANCE
    Cash: 1.0710 Total Val: 1.0710 PnL: 0.0710 Returns: 0.0710
BALANCE
    BTC - {'free': 1.1420599999999999, 'used': 0.0, 'total': 1.1420599999999999}
    ETH - {'free': 0.0, 'used': 0.0, 'total': 0.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quantity': 0.0, 'cost_price': 0.0, 'latest_price': 0.0888}
METRICS
    SMA: 5.00
    RSI: 10.00
---------------------------------------
Epoch 147 - Timestep: 2018-01-16T02:30:00
---------------------------------------
OHLCV
    O: 0.0945 | C: 0.0949 | V: 2114.9 | T: 2018-01-16T02:30:00
ORDERS
    1: ETH/BTC | MARKET_SELL | Price: 0.0000 | Quantity: 1.0000
PERFORMANCE
    Cash: 0.9766 Total Val: 1.0710 PnL: 0.0710 Returns: 0.0710
BALANCE
    BTC - {'free': 0.95320799999999994, 'used': 0.0, 'total': 0.95320799999999994}
    ETH - {'free': 2.0, 'used': 0.0, 'total': 2.0}
POSITIONS
     {'asset': 'ETH/BTC', 'quan

Order Places Response {'id': '0ddb25f221f54cfda0552128166c3b0f', 'symbol': 'ETH/BTC', 'price': 0.094425999999999996, 'quantity': 1.0, 'type': 'limit', 'side': 'buy', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:41.912380'}
Order dictionary {'id': '0ddb25f221f54cfda0552128166c3b0f', 'symbol': 'ETH/BTC', 'price': 0.094425999999999996, 'quantity': 1.0, 'type': 'limit', 'side': 'buy', 'filled': 0.0, 'status': 'FILLED', 'fee': {}, 'created_time': '2018-01-17T06:08:41.912380'}
Order fetched {
    "id": null,
    "exchange_id": "paper",
    "exchange_order_id": "0ddb25f221f54cfda0552128166c3b0f",
    "price": 0.094426,
    "quantity": 1.0,
    "filled_quantity": 0.0,
    "order_type": "LIMIT_BUY",
    "status": "FILLED",
    "created_time": "2018-01-17T06:08:41.912380",
    "opened_time": null,
    "filled_time": null,
    "canceled_time": null,
    "fee": {},
    "retries": 0,
    "symbol": "ETH/BTC"
}
Equal? True
ORDER {
    "id": "c174835dda7d4ca38b420865

## Simulate Orders

In [ ]:
cash = 1.0
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_ccxt_based_paper_exchange(balance, c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.ETH, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_simulate'

In [ ]:
runner.simulate(experiment_name, exchange, balance, portfolio, feed, strategy)

## Live Trade

In [ ]:
def live(name, exchange, balance, portfolio, feed, strategy):
    print("WARNING LIVE TRADING!!!!")
    '''
    exp_name = name of your current experiment (multiple runs per strategy)
    '''
    # Where we will save the record
    root = os.path.join(cfg.DATA_DIR, name)

    # This can be retrieved from the user's global config
    store = DATA_STORES[cfg.DATA_STORE](root=root)

    config = {
        'experiment': name, 
        'strategy': strategy.name,
    }
    record = Record(
        config=config,
        portfolio=portfolio,
        balance=balance,
        store=store
    )
    ctx = Context(
        exchange=exchange,
        feed=feed,
        record=record
    )
    feed.initialize()
    
    while True:
        row = feed.next()
        
        if row is not None:
            orders = strategy.process(row, ctx)

            # TODO: Cancelling orders
            # should we auto-cancel any outstanding orders
            # or should we leave this decision up to the Strategy?
            order_manager.cancel_orders(exchange, orders['cancel_ids'])

            # Returns both FILLED and PENDING orders
            # TODO: Order manager handles mapping from Exchange JSON
            # Particularly order types like CLOSED --> FILLED,
            # And OPEN vs PENDING <-- check the 'quantity' vs 'filled' amounts
            orders = order_manager.place_orders(exchange, orders['orders'])
            filled_orders = order_manager.get_filled_orders(orders)

            # Portfolio needs to know about new filled orders
            record.portfolio.update(filled_orders)

            # Record needs to know about all new orders
            for order in orders:
                record.orders[order.id] = order

            # Update Virtual Balance
            # exchange balance may be impacted by external trading
            for order in filled_orders:
                record.balance.update_by_order(order)

            record.save()
        
        time.sleep(30)
            
    return record

In [ ]:
cash = .04
balance = Balance(
    cash_currency=c.BTC,
    starting_cash=cash
)
exchange = load_exchange(c.BINANCE)
perf_tracker = PerformanceTracker(
    starting_cash=cash, 
    timeframe=Timeframe.ONE_MIN
)
portfolio = Portfolio(
    starting_cash=cash,
    perf_tracker=perf_tracker, # option to override, otherwise default
    positions=None # option to override with existing positions
)
assets = [Asset(c.XRP, c.BTC)]
feed = ExchangeDataFeed(
    exchange=exchange,
    assets=assets,
    timeframe=Timeframe.ONE_MIN,
    start=datetime.datetime.utcnow(),
    end=None
)
strategy = SimpleStrategy()
experiment_name = 'default_live'

In [ ]:
live(experiment_name, exchange, balance, portfolio, feed, strategy)